<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/Ro_large.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-ene6jd_d
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-ene6jd_d
  Resolved https://github.com/huggingface/transformers to commit 25b0f2033ba23e354ef2f665764248fcbb3f49ba
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [13]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [14]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [15]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "ro", split="test")
#column_name = "audio, normalized_text"

In [16]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("large")

In [17]:
model = whisper.load_model("large")

100%|█████████████████████████████████████| 2.88G/2.88G [00:36<00:00, 85.1MiB/s]


In [20]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [21]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="ro")
    return result["text"]

In [22]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [23]:
from jiwer import wer
from collections import Counter

def calculate_precision_recall_f1(ground_truth, hypothesis):
    ground_truth_words = set(ground_truth.split())
    hypothesis_words = set(hypothesis.split())

    true_positives = ground_truth_words.intersection(hypothesis_words)
    false_positives = hypothesis_words - ground_truth_words
    false_negatives = ground_truth_words - hypothesis_words

    precision = len(true_positives) / (len(true_positives) + len(false_positives)) if true_positives or false_positives else 0
    recall = len(true_positives) / (len(true_positives) + len(false_negatives)) if true_positives or false_negatives else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall else 0

    return precision, recall, f1_score

In [24]:
total_wer = 0
total_precision = 0
total_recall = 0
total_f1 = 0
count = 0

for sample in voxpopuli_data:
    audio_file_path = sample['audio']['path']
    ground_truth_text = sample['normalized_text']

    # Transcribe the audio
    transcription = transcribe_audio(audio_file_path)


 # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)
    precision, recall, f1_score = calculate_precision_recall_f1(ground_truth_text, transcription)

    total_wer += sample_wer
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score
    count += 1
    print(f"Processed {count} files, WER: {sample_wer}, Precision: {precision}, Recall: {recall}, F1: {f1_score}")

# Calculate average metrics
average_wer = total_wer / count
average_precision = total_precision / count
average_recall = total_recall / count
average_f1 = total_f1 / count

print(f"Average WER: {average_wer}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1-Score: {average_f1}")

Processed 1 files, WER: 0.16666666666666666, Precision: 0.8260869565217391, Recall: 0.8260869565217391, F1: 0.8260869565217391
Processed 2 files, WER: 0.2916666666666667, Precision: 0.6470588235294118, Recall: 0.6470588235294118, F1: 0.6470588235294118
Processed 3 files, WER: 0.2, Precision: 0.8, Recall: 0.8571428571428571, F1: 0.8275862068965518
Processed 4 files, WER: 0.30434782608695654, Precision: 0.6842105263157895, Recall: 0.6842105263157895, F1: 0.6842105263157895
Processed 5 files, WER: 0.19047619047619047, Precision: 0.8, Recall: 0.8, F1: 0.8000000000000002
Processed 6 files, WER: 0.3548387096774194, Precision: 0.7037037037037037, Recall: 0.6785714285714286, F1: 0.6909090909090909
Processed 7 files, WER: 0.125, Precision: 0.875, Recall: 0.875, F1: 0.875
Processed 8 files, WER: 0.1875, Precision: 0.7857142857142857, Recall: 0.7857142857142857, F1: 0.7857142857142857
Processed 9 files, WER: 0.2, Precision: 0.7777777777777778, Recall: 0.7777777777777778, F1: 0.7777777777777778
Pr